* [LiteLLM - Getting Started](https://docs.litellm.ai/docs/)
* [LiteLLM Cookbook](https://github.com/BerriAI/litellm/tree/main/cookbook)

In [5]:
import os
import json
import operator
from typing import Annotated, List, Literal, Optional

from pydantic import BaseModel, Field
from tavily import TavilyClient
from langgraph.graph import StateGraph, START, END
from litellm import completion

In [32]:
path_to_openai_key:str = os.path.expanduser('~/.openai/api_key')
with open(path_to_openai_key, 'r', encoding='utf-8') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

path_to_tavily_key:str = os.path.expanduser('~/.tavily/api_key')
with open(path_to_tavily_key, 'r', encoding='utf-8') as file:
    os.environ["TAVILY_API_KEY"] = file.read().strip()

# Batch

In [44]:
try:
    # responses is generator
    response= completion(
        model="openai/gpt-4o",
        messages=[{ "content": "Hello, how are you?","role": "user"}],
        stream=False
    )

except litellm.AuthenticationError as e:
    print(f"Authentication failed: {e}")
except litellm.RateLimitError as e:
    print(f"Rate limited: {e}")
except litellm.APIError as e:
    print(f"API error: {e}")

print(json.dumps(response.model_dump(), indent=4, default=str))

{
    "id": "chatcmpl-Cq5XVwiBd3qSRYEx9Rvcaxcre08YX",
    "created": 1766530385,
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "system_fingerprint": "fp_deacdd5f6f",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "message": {
                "content": "Hello! I'm just a machine, but I'm here and ready to help you with whatever you need. How can I assist you today?",
                "role": "assistant",
                "tool_calls": null,
                "function_call": null,
                "provider_specific_fields": {
                    "refusal": null
                },
                "annotations": []
            },
            "provider_specific_fields": {}
        }
    ],
    "usage": {
        "completion_tokens": 27,
        "prompt_tokens": 13,
        "total_tokens": 40,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
          

# Streaming

In [13]:
try:
    # responses is generator
    responses = completion(
        model="openai/gpt-4o",
        messages=[{ "content": "Hello, how are you?","role": "user"}],
        stream=True
    )

    # Iterate through the generator
    for chunk in responses:
        # Extract the content delta
        content = chunk.choices[0].delta.content
        
        if content:
            # print(..., end="", flush=True) creates the typewriter effect
            print(content, end="", flush=True)
            
    print() # Print a newline when the stream finishes    
except litellm.AuthenticationError as e:
    print(f"Authentication failed: {e}")
except litellm.RateLimitError as e:
    print(f"Rate limited: {e}")
except litellm.APIError as e:
    print(f"API error: {e}")

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [45]:
SearchTool.__doc__

'Search the web for current events, news, or deep research.'

In [39]:
# 1. SETUP
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# 2. STATE DEFINITION (Pydantic)
class AgentState(BaseModel):
    messages: Annotated[List[dict], operator.add] = Field(default_factory=list)

# 3. ADVANCED SCHEMA DEFINITION
class SearchTool(BaseModel):
    """Search the web for current events, news, or deep research."""
    query: str = Field(description="The search query to look up")
    
    topic: Literal["general", "news", "finance"] = Field(
        default="general",
        description="Category of search. Use 'news' for current events/politics, 'finance' for market data."
    )
    
    search_depth: Literal["basic", "advanced"] = Field(
        default="basic",
        description="Use 'basic' for quick facts. Use 'advanced' for complex queries needing more context."
    )
    
    time_range: Optional[Literal["day", "week", "month", "year"]] = Field(
        default=None,
        description="Filter results by publication date. Especially useful with topic='news'."
    )
    
    max_results: int = Field(
        default=5, ge=1, le=10,
        description="Number of search results to return."
    )

tools_schema = [{
    "type": "function",
    "function": {
        "name": "SearchTool",
        "description": SearchTool.__doc__,
        "parameters": SearchTool.model_json_schema()
    }
}]

# 4. NODES
def call_model(state: AgentState):
    response = completion(
        model="gpt-4o", 
        messages=state.messages, 
        tools=tools_schema
    )
    return {"messages": [response.choices[0].message.dict()]}

def call_tool(state: AgentState):
    last_msg = state.messages[-1]
    tool_call = last_msg["tool_calls"][0]
    
    # Extract arguments - these now include topic, search_depth, etc.
    args = json.loads(tool_call["function"]["arguments"])
    
    # 5. DYNAMIC TAVILY CALL
    # Passing **args allows Tavily to receive topic, time_range, etc. automatically
    search_data = tavily.search(**args)
    
    content = "\n".join([
        f"Source: {r['title']}\nURL: {r['url']}\nContent: {r['content']}\n---" 
        for r in search_data['results']
    ])
    
    return {"messages": [{
        "role": "tool",
        "tool_call_id": tool_call["id"],
        "name": "SearchTool",
        "content": content
    }]}

# 6. ROUTER & GRAPH
def router(state: AgentState) -> Literal["call_tool", "__end__"]:
    last_msg = state.messages[-1]
    return "call_tool" if last_msg.get("tool_calls") else "__end__"

builder = StateGraph(AgentState)
builder.add_node("llm", call_model)
builder.add_node("call_tool", call_tool)
builder.add_edge(START, "llm")
builder.add_conditional_edges("llm", router)
builder.add_edge("call_tool", "llm")
app = builder.compile()

In [40]:
# 7. EXECUTION
inputs = {"messages": [{"role": "user", "content": "What are the top news headlines in AI from the last week?"}]}

for event in app.stream(inputs, stream_mode="values"):
    last_msg = event["messages"][-1]
    
    # Check if it's a dictionary (from our nodes) or a LiteLLM object
    role = last_msg.get("role", "assistant").upper() if isinstance(last_msg, dict) else last_msg.role.upper()
    
    # Robust display logic
    tool_calls = last_msg.get("tool_calls") if isinstance(last_msg, dict) else getattr(last_msg, "tool_calls", None)

    if tool_calls:
        # tool_calls[0] is an object, but function.arguments is a string
        first_call = tool_calls[0]
        # Handle both object-style and dict-style access
        args_str = first_call["function"]["arguments"] if isinstance(first_call, dict) else first_call.function.arguments
        params = json.loads(args_str)
        print(f"\n--- {role} (Calling Search with params: {params}) ---")
    else:
        content = last_msg.get("content") if isinstance(last_msg, dict) else getattr(last_msg, "content", "")
        print(f"\n--- {role} ---")
        if content:
            print(content)
        else:
            print("Processing...")


--- USER ---
What are the top news headlines in AI from the last week?


/var/folders/_4/8v285hqs45xfzk0l1nlr3yq40000gn/T/ipykernel_22303/2486168244.py:56: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return {"messages": [response.choices[0].message.dict()]}



--- ASSISTANT (Calling Search with params: {'query': 'AI news', 'topic': 'news', 'search_depth': 'basic', 'time_range': 'week', 'max_results': 5}) ---

--- TOOL ---
Source: Investors Focus on AI Startups for Future Growth - Startup Ecosystem Canada
URL: https://www.startupecosystem.ca/news/investors-focus-on-ai-startups-for-future-growth/
Content: ## Investors Focus on AI Startups for Future Growth. At TechCrunch Disrupt, investors emphasized their strong interest in artificial intelligence, highlighting the importance of startups demonstrating resilience and domain expertise in the rapidly evolving AI market. AI startupsEnterprise AI solutionsGlobal startup newsTechCrunch Disruptventure capital. acquisition AI AI development AI infrastructure AI innovation AI integration AI investment AI models AI technology Amazon Anthropic Apple Artificial Intelligence Autonomous vehicles Canada founder news Canada startup news ChatGPT cryptocurrency Cybersecurity DeepSeek Electric Vehicles Elon Mu